In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import seaborn as sns
import seaborn.objects as so
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict



from src.DataProcessor import DataProcessor


In [11]:
PATH = r'/Users/michaelschaid/GitHub/credit_modeling/data/loans_2007.csv'
DATA = (DataProcessor(PATH, target = 'loan_status')
        .load_data()
        .clean_data()
        .processes_dummies()
        .extract_features_and_target()
        )

In [18]:

lr = LogisticRegression()
X = DATA.feature_data
y = DATA.target_data
y_pred = cross_val_predict(lr,X,y, cv=3)
predictions = pd.Series(y_pred)


/Users/michaelschaid/opt/anaconda3/envs/loans/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [16]:
predictions.value_counts()

1    36923
0       67
dtype: int64

In [34]:
def calculate_rates(y_pred: pd.Series, y_true: pd.Series)-> float:
    """## Summary
    takes in predicted and acutal values, calculate true postive, true neg, false pos, and false neg
    returns true postve rate and false postve rate
    `

    Args:
        y_pred (pd.Series): predicted y values
        y_true (pd.Seres): true y values

    Returns:
        tuple: true postve rate:float and false postve rate: float
    """
    true_pos = sum((y_pred == 1) & (y_true == 1))
    false_pos = sum((y_pred == 1) & (y_true == 0))
    true_neg = sum((y_pred == 0) & (y_true == 0))
    false_neg = sum((y_pred == 0) & (y_true == 1))
    
    false_pos_rate = false_pos/(false_pos + true_neg)
    true_pos_rate = true_pos/(true_pos + false_neg)
    
    return true_pos_rate, false_pos_rate
    
    
tpr, fpr = calculate_rates(predictions, y)
print(f'true positive rate: {tpr}, \n false postive rate {fpr}')

true positive rate: 0.9980286678472384, 
 false postive rate 0.9991956565453448


Model is predictingly nearlly all ones, due to imbalance in the class
We will test both: 

*  oversampling and undersampling to ensure that the classifier gets input that has a balanced number of each class.
  
* Tell the classifier to penalize misclassifications of the less prevalent class more than the other class.
